## Phase 3: Feature Engineering and Data Generation

This notebook orchestrates the entire data generation process. It combines the solve function from Phase 2 and the extract_features function from Phase 3 to create the final dataset for training our meta-learning model.

Process:
1.  Setup: Import libraries and connect to your real solver/feature extractor functions.
2.  Define Instances: Create sample problem instances to simulate a benchmark suite.
3.  Execution Loop: Iterate through each instance and each algorithm.
4.  Data Collection: For each run, extract features and record performance metrics.
5.  Save Dataset: Combine all results into a single pandas DataFrame and save to a CSV file.

In [1]:
import pandas as pd
import numpy as np
import os
import time
import json
from collections import namedtuple
import sys

# Add the project's root directory to the Python path to find the 'src' module
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.main import solve
from src.features.extractor import extract_features

print("Setup Complete. Real solver and feature extractor are imported.")

Setup Complete. Real solver and feature extractor are imported.


### Step 2: Define and Create Benchmark Instances

This cell creates a few sample problem instance files in the /data directory to simulate a real benchmark suite. If these files already exist, it will skip them.

In [2]:
DATA_DIR = '../data'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

def create_instance_file(filename, data):
    filepath = os.path.join(DATA_DIR, filename)
    if not os.path.exists(filepath):
        with open(filepath, 'w') as f:
            json.dump(data, f)
        print(f"Created {filename}")
    else:
        print(f"{filename} already exists.")

# --- Create a sample TSP instance ---
tsp_instance = {
    'type': 'TSP',
    'distance_matrix': [
        [0, 29, 20, 21, 16, 31, 100, 12, 4, 31, 18],
        [29, 0, 15, 29, 28, 40, 72, 21, 29, 41, 12],
        [20, 15, 0, 15, 14, 25, 81, 9, 23, 27, 13],
        [21, 29, 15, 0, 4, 12, 92, 12, 25, 13, 25],
        [16, 28, 14, 4, 0, 16, 94, 9, 20, 16, 22],
        [31, 40, 25, 12, 16, 0, 95, 24, 36, 3, 37],
        [100, 72, 81, 92, 94, 95, 0, 90, 101, 99, 84],
        [12, 21, 9, 12, 9, 24, 90, 0, 15, 25, 13],
        [4, 29, 23, 25, 20, 36, 101, 15, 0, 35, 18],
        [31, 41, 27, 13, 16, 3, 99, 25, 35, 0, 38],
        [18, 12, 13, 25, 22, 37, 84, 13, 18, 38, 0]
    ]
}
create_instance_file('tsp_instance_01.json', tsp_instance)

# --- Create a sample Knapsack instance ---
kp_instance = {
    'type': 'KP',
    'values': [48, 30, 42, 36, 22, 43, 18, 24, 36, 29, 30, 25, 19, 41, 34, 32, 27, 24, 18],
    'weights': [10, 30, 12, 22, 12, 20, 9, 9, 18, 20, 25, 18, 7, 16, 24, 21, 21, 32, 9],
    'capacity': 150
}
create_instance_file('kp_instance_01.json', kp_instance)

tsp_instance_01.json already exists.
kp_instance_01.json already exists.


In [3]:
# --- Add this to a new cell in your notebook ---

print("\\nGenerating additional random instances...")

# Generate 50 random TSP instances
for i in range(2, 52):  # Start from 2 to not overwrite your first one
    num_cities = np.random.randint(10, 30) # Random size between 10 and 30
    dist_matrix = np.random.randint(10, 200, size=(num_cities, num_cities))
    np.fill_diagonal(dist_matrix, 0)
    dist_matrix = (dist_matrix + dist_matrix.T) // 2 # Make it symmetric
    
    instance_data = {
        'type': 'TSP',
        'distance_matrix': dist_matrix.tolist()
    }
    create_instance_file(f'tsp_instance_{i:02d}.json', instance_data)

# Generate 50 random Knapsack instances
for i in range(2, 52):
    num_items = np.random.randint(15, 40)
    values = np.random.randint(10, 100, size=num_items).tolist()
    weights = np.random.randint(5, 50, size=num_items).tolist()
    # Make capacity "tight" - between 30% and 70% of total weight
    capacity = int(sum(weights) * np.random.uniform(0.3, 0.7))
    
    instance_data = {
        'type': 'KP',
        'values': values,
        'weights': weights,
        'capacity': capacity
    }
    create_instance_file(f'kp_instance_{i:02d}.json', instance_data)

print("\\nInstance generation complete.")

\nGenerating additional random instances...
Created tsp_instance_02.json
Created tsp_instance_03.json
Created tsp_instance_04.json
Created tsp_instance_05.json
Created tsp_instance_06.json
Created tsp_instance_07.json
Created tsp_instance_08.json
Created tsp_instance_09.json
Created tsp_instance_10.json
Created tsp_instance_11.json
Created tsp_instance_12.json
Created tsp_instance_13.json
Created tsp_instance_14.json
Created tsp_instance_15.json
Created tsp_instance_16.json
Created tsp_instance_17.json
Created tsp_instance_18.json
Created tsp_instance_19.json
Created tsp_instance_20.json
Created tsp_instance_21.json
Created tsp_instance_22.json
Created tsp_instance_23.json
Created tsp_instance_24.json
Created tsp_instance_25.json
Created tsp_instance_26.json
Created tsp_instance_27.json
Created tsp_instance_28.json
Created tsp_instance_29.json
Created tsp_instance_30.json
Created tsp_instance_31.json
Created tsp_instance_32.json
Created tsp_instance_33.json
Created tsp_instance_34.json

### Step 3 & 4: The Experiment Loop

This is the core of the notebook. We loop through all problem instances and all relevant algorithms, generating the data.

In [4]:
all_records = []
algorithm_map = {
    'TSP': ['GA', 'SA'], # Add 'CP-SAT' here if you have a TSP model for it
    'KP': ['B&B']  # Add 'GA', 'SA', 'CP-SAT' if you have KP models for them
}

# Discover instance files in the data directory
instance_files = [f for f in os.listdir(DATA_DIR) if f.endswith('.json')]

print(f"Starting experiment on {len(instance_files)} instances...")

for filename in instance_files:
    filepath = os.path.join(DATA_DIR, filename)
    with open(filepath, 'r') as f:
        instance_data = json.load(f)
    
    problem_type = instance_data.get('type')
    if not problem_type:
        print(f"Skipping {filename}: missing 'type' key.")
        continue

    print(f"\nProcessing {filename} (Type: {problem_type})...")

    # 1. Extract features ONCE per instance
    features = extract_features(instance_data, problem_type)
    features['instance_id'] = filename
    features['problem_type'] = problem_type
        
    # 2. Run EACH relevant algorithm on the instance
    for algo in algorithm_map.get(problem_type, []):
        print(f"  -> Running {algo}...")
        try:
            solution, metrics = solve(instance_data, algo)
            
            record = features.copy()
            record['algorithm'] = algo
            record['runtime_seconds'] = metrics.get('runtime', 0.0)
            record['solution_quality'] = metrics.get('objective_value', 0.0)
            
            all_records.append(record)
            print(f"     ...Done. Runtime={record['runtime_seconds']:.4f}s, Quality={record['solution_quality']}")
        except Exception as e:
            print(f"     ...Failed to run {algo} on {filename}: {e}")

print("\nExperiment finished!")

2025/10/06 01:10:10 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: BaseGA(epoch=200, pop_size=100, pc=0.9, pm=0.05)
2025/10/06 01:10:10 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 1, Current best: 1296.0, Global best: 1174.0, Runtime: 0.01058 seconds
2025/10/06 01:10:10 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 2, Current best: 1184.0, Global best: 1174.0, Runtime: 0.01237 seconds
2025/10/06 01:10:10 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 3, Current best: 1220.0, Global best: 1174.0, Runtime: 0.01330 seconds
2025/10/06 01:10:10 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 4, Current best: 1296.0, Global best: 1174.0, Runtime: 0.01135 seconds
2025/10/06 01:10:10 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 5, Current best: 1249.0, Global best: 1174.0, Runtime: 0.06869 seconds
2025/10/06 01:10:10 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Ep

Starting experiment on 102 instances...

Processing tsp_instance_36.json (Type: TSP)...
  -> Running GA...


2025/10/06 01:10:10 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 8, Current best: 1163.0, Global best: 1163.0, Runtime: 0.01133 seconds
2025/10/06 01:10:10 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 9, Current best: 1147.0, Global best: 1147.0, Runtime: 0.01118 seconds
2025/10/06 01:10:10 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 10, Current best: 1120.0, Global best: 1120.0, Runtime: 0.01100 seconds
2025/10/06 01:10:10 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 11, Current best: 1153.0, Global best: 1120.0, Runtime: 0.01089 seconds
2025/10/06 01:10:10 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 12, Current best: 1211.0, Global best: 1120.0, Runtime: 0.01136 seconds
2025/10/06 01:10:10 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 13, Current best: 1158.0, Global best: 1120.0, Runtime: 0.01120 seconds
2025/10/06 01:10:10 AM, INFO, mealpy

     ...Done. Runtime=2.4811s, Quality=943
  -> Running SA...


 Temperature        Energy    Accept   Improve     Elapsed   Remaining
2025/10/06 01:10:19 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: BaseGA(epoch=200, pop_size=100, pc=0.9, pm=0.05)
2025/10/06 01:10:19 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 1, Current best: 1132.0, Global best: 1132.0, Runtime: 0.01029 seconds
2025/10/06 01:10:19 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 2, Current best: 1280.0, Global best: 1132.0, Runtime: 0.01052 seconds
2025/10/06 01:10:19 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 3, Current best: 1156.0, Global best: 1132.0, Runtime: 0.01031 seconds
2025/10/06 01:10:19 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 4, Current best: 1283.0, Global best: 1132.0, Runtime: 0.01053 seconds
2025/10/06 01:10:19 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 5, Current best: 1227.0, Global best: 1132.0, Runtime: 0.01042 seconds
2025/10/06 01

...

     ...Done. Runtime=6.8089s, Quality=1270

Processing kp_instance_33.json (Type: KP)...
  -> Running B&B...
     ...Failed to run B&B on kp_instance_33.json: module 'src.solvers.branch_and_bound_solver' has no attribute 'solve_knapsack_with_bnb'

Processing tsp_instance_37.json (Type: TSP)...
  -> Running GA...


2025/10/06 01:18:38 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 17, Current best: 2023.0, Global best: 1909.0, Runtime: 0.01141 seconds
2025/10/06 01:18:38 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 18, Current best: 2022.0, Global best: 1909.0, Runtime: 0.01092 seconds
2025/10/06 01:18:38 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 19, Current best: 1887.0, Global best: 1887.0, Runtime: 0.01095 seconds
2025/10/06 01:18:38 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 20, Current best: 2006.0, Global best: 1887.0, Runtime: 0.01062 seconds
2025/10/06 01:18:38 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 21, Current best: 2217.0, Global best: 1887.0, Runtime: 0.01154 seconds
2025/10/06 01:18:38 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 22, Current best: 2115.0, Global best: 1887.0, Runtime: 0.01070 seconds
2025/10/06 01:18:38 AM, INFO, meal

     ...Done. Runtime=2.3554s, Quality=1825
  -> Running SA...


 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     4.50000       1169.00     4.16%     0.02%     0:00:06     0:00:00

     ...Done. Runtime=6.6428s, Quality=1078

Experiment finished!


In [5]:
if all_records:
    df = pd.DataFrame(all_records)
    
    # Define column order for clarity
    meta_cols = ['instance_id', 'problem_type', 'algorithm']
    result_cols = ['runtime_seconds', 'solution_quality']
    feature_cols = [col for col in df.columns if col not in meta_cols + result_cols]
    
    df = df[meta_cols + result_cols + feature_cols]
    
    output_path = os.path.join(DATA_DIR, 'generated_dataset.csv')
    df.to_csv(output_path, index=False)
    
    print(f"\nSuccessfully created dataset with {len(df)} rows and {len(df.columns)} columns.")
    print(f"Saved to: {output_path}")
    display(df.head())
else:
    print("\nNo records were generated. Cannot create dataset.")


Successfully created dataset with 102 rows and 14 columns.
Saved to: ../data/generated_dataset.csv


,instance_id,problem_type,algorithm,runtime_seconds,solution_quality,instance_size,data_mean,data_std_dev,data_skewness,data_kurtosis,data_min,data_max,data_q1,data_q3
0,tsp_instance_36.json,TSP,GA,2.481141,943,15,105.904762,40.505032,0.214874,-1.152037,33,183,75.00,142.0
1,tsp_instance_36.json,TSP,SA,6.684113,829,15,105.904762,40.505032,0.214874,-1.152037,33,183,75.00,142.0
2,tsp_instance_25.json,TSP,GA,2.368710,907,16,99.175000,40.778193,0.244822,-0.704596,19,196,66.75,127.5
3,tsp_instance_25.json,TSP,SA,6.464394,762,16,99.175000,40.778193,0.244822,-0.704596,19,196,66.75,127.5
4,tsp_instance_28.json,TSP,GA,2.507938,1643,24,102.898551,40.599991,0.068628,-0.742216,13,191,73.00,133.0
